## A beautifull way to work with data

This is an API proposal to access data.

Dataframes would have rows and columns. 

* To access columns just use df.cols()
* To access rows just use df.rows()
* I/O operations to load and save data are in Optimus. op.load.csv(). op.save.csv()

Easy and simple

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from optimus import *

from pyspark.sql.session import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, IntegerType, ArrayType

sc = SparkSession.builder.getOrCreate()

In [3]:
# Create optimus
op = Optimus(sc)

Using a created Spark Session...
Done.


## Create dataframe
### Spark

This is ugly:

```
val someData = Seq(
  Row(8, "bat"),
  Row(64, "mouse"),
  Row(-27, "horse")
)

val someSchema = List(
  StructField("number", IntegerType, true),
  StructField("word", StringType, true)
)

val someDF = spark.createDataFrame(
  spark.sparkContext.parallelize(someData),
  StructType(someSchema)
)```

In [65]:
# Thanks Mr Powers
df = op.create.df([
                ("  I like     fish  ", 1, "dog", "housé", "cat-car", "a"),
                ("    zombies", 2, "cat", "tv", "dog-tv", "b"),
                ("simpsons   cat lady", 2, "frog", "table","eagle-tv-plus","1"),
                (None, 3, "eagle", "glass", "lion-pc", "c")
            ],
            [
                ("words", "str", True),
                ("num", "int", True),
                ("animals", "str", True),
                ("thing", StringType(), True),
                ("two strings", StringType(), True),
                ("filter", StringType(), True)
            ])

df.show()

+-------------------+---+-------+-----+-------------+------+
|              words|num|animals|thing|  two strings|filter|
+-------------------+---+-------+-----+-------------+------+
|  I like     fish  |  1|    dog|housé|      cat-car|     a|
|            zombies|  2|    cat|   tv|       dog-tv|     b|
|simpsons   cat lady|  2|   frog|table|eagle-tv-plus|     1|
|               null|  3|  eagle|glass|      lion-pc|     c|
+-------------------+---+-------+-----+-------------+------+



## Create Columns
### Spark
* You can not create multiple columns at the same time
* You need to use the lit function. lit???

### Pandas
* Similiar behavior


In [4]:
df = df.cols().create("new_col_1", 1)
df.show()

+-------------------+---+-------+-----+---------+
|              words|num|animals|thing|new_col_1|
+-------------------+---+-------+-----+---------+
|  I like     fish  |  1|    dog|housé|        1|
|            zombies|  2|    cat|   tv|        1|
|simpsons   cat lady|  2|   frog|table|        1|
|               null|  3|  eagle|glass|        1|
+-------------------+---+-------+-----+---------+



In [5]:
sf = df.cols().create([
    ("new_col_2", 2.22),
    ("new_col_3", lit(3)),
    ("new_col_4", "test"),
    ("new_col_5", df['num']*2)
    ])

df.show()

+-------------------+---+-------+-----+---------+
|              words|num|animals|thing|new_col_1|
+-------------------+---+-------+-----+---------+
|  I like     fish  |  1|    dog|housé|        1|
|            zombies|  2|    cat|   tv|        1|
|simpsons   cat lady|  2|   frog|table|        1|
|               null|  3|  eagle|glass|        1|
+-------------------+---+-------+-----+---------+



## Select columns
### Spark
* You can not select columns by string and index at the same time

### Pandas
* You can not select columns by string and index at the same time

In [7]:
columns = ["words", 1, "animals", 3]
df.cols().select(columns).show()

+-------------------+---+-------+-----+
|              words|num|animals|thing|
+-------------------+---+-------+-----+
|  I like     fish  |  1|    dog|housé|
|            zombies|  2|    cat|   tv|
|simpsons   cat lady|  2|   frog|table|
|               null|  3|  eagle|glass|
+-------------------+---+-------+-----+



In [8]:
df.cols().select(regex = "n.*").show()

+---+---------+
|num|new_col_1|
+---+---------+
|  1|        1|
|  2|        1|
|  2|        1|
|  3|        1|
+---+---------+



## Rename Column
### Spark
You can not rename multiple columns using Spark Vanilla API


### Pandas
Almost the same behavior that pandas

In [9]:
df.cols().rename([('num','number')]).show()

+-------------------+------+-------+-----+---------+
|              words|number|animals|thing|new_col_1|
+-------------------+------+-------+-----+---------+
|  I like     fish  |     1|    dog|housé|        1|
|            zombies|     2|    cat|   tv|        1|
|simpsons   cat lady|     2|   frog|table|        1|
|               null|     3|  eagle|glass|        1|
+-------------------+------+-------+-----+---------+



## Rename
### Spark
* You can not rename multiple columns

### Pandas
* Almost the same behavior https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.rename.html

In [10]:
df.cols().rename(func = str.lower).show()

+-------------------+---+-------+-----+---------+
|              words|num|animals|thing|new_col_1|
+-------------------+---+-------+-----+---------+
|  I like     fish  |  1|    dog|housé|        1|
|            zombies|  2|    cat|   tv|        1|
|simpsons   cat lady|  2|   frog|table|        1|
|               null|  3|  eagle|glass|        1|
+-------------------+---+-------+-----+---------+



In [12]:
df.cols().rename(func = str.upper).show()

+-------------------+---+-------+-----+---------+
|              WORDS|NUM|ANIMALS|THING|NEW_COL_1|
+-------------------+---+-------+-----+---------+
|  I like     fish  |  1|    dog|housé|        1|
|            zombies|  2|    cat|   tv|        1|
|simpsons   cat lady|  2|   frog|table|        1|
|               null|  3|  eagle|glass|        1|
+-------------------+---+-------+-----+---------+



In [13]:
df.cols().rename(columns =[('num','number')]).show()

+-------------------+------+-------+-----+---------+
|              words|number|animals|thing|new_col_1|
+-------------------+------+-------+-----+---------+
|  I like     fish  |     1|    dog|housé|        1|
|            zombies|     2|    cat|   tv|        1|
|simpsons   cat lady|     2|   frog|table|        1|
|               null|     3|  eagle|glass|        1|
+-------------------+------+-------+-----+---------+



## Cast a columns

### Spark
* Can not cast multiple columns

### Pandas
This is a opinionated way to handle column casting. 
One of the first thing that every data cleaning process need to acomplish is define a data dictionary.
Because of that we prefer to create a tuple like this:

df.cols().cast(
[("words","str"),
("num","int"),
("animals","float"),
("thing","str")]
)

instead of pandas

pd.Series([1], dtype='int32')
pd.Series([2], dtype='string')

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.astype.html

In [14]:
df.cols().cast([("num", "string")])

DataFrame[words: string, num: string, animals: string, thing: string, new_col_1: int]

## Keep columns
### Spark
* You can not remove multiple columns

### Pandas
* Handle in pandas with drop


In [15]:
df.cols().keep("num").show()

+---+
|num|
+---+
|  1|
|  2|
|  2|
|  3|
+---+



## Move columns
### Spark
Do not exist in spark

### Pandas
Do not exist in pandas

In [16]:
df.cols().move("words", "thing", "after").show()

+---+-------+-----+-------------------+---------+
|num|animals|thing|              words|new_col_1|
+---+-------+-----+-------------------+---------+
|  1|    dog|housé|  I like     fish  |        1|
|  2|    cat|   tv|            zombies|        1|
|  2|   frog|table|simpsons   cat lady|        1|
|  3|  eagle|glass|               null|        1|
+---+-------+-----+-------------------+---------+



## Sorting Columns
### Spark
You can not sort columns using Spark Vanilla API 

### Pandas
Similar to pandas
http://pandas.pydata.org/pandas-docs/version/0.19/generated/pandas.DataFrame.sort_values.html#pandas.DataFrame.sort_values

In [17]:
df.cols().sort().show()

+-------+---------+---+-----+-------------------+
|animals|new_col_1|num|thing|              words|
+-------+---------+---+-----+-------------------+
|    dog|        1|  1|housé|  I like     fish  |
|    cat|        1|  2|   tv|            zombies|
|   frog|        1|  2|table|simpsons   cat lady|
|  eagle|        1|  3|glass|               null|
+-------+---------+---+-----+-------------------+



In [18]:
df.cols().sort(reverse = True).show()

+-------------------+-----+---+---------+-------+
|              words|thing|num|new_col_1|animals|
+-------------------+-----+---+---------+-------+
|  I like     fish  |housé|  1|        1|    dog|
|            zombies|   tv|  2|        1|    cat|
|simpsons   cat lady|table|  2|        1|   frog|
|               null|glass|  3|        1|  eagle|
+-------------------+-----+---+---------+-------+



## Drop columns
### Spark 
* You can not delete multiple colums

### Pandas
* Almost the same as pandas
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html

In [19]:
df2 = df.cols().drop("num")
df2 = df.cols().drop(["num","words"])
df2.show()

+-------+-----+---------+
|animals|thing|new_col_1|
+-------+-----+---------+
|    dog|housé|        1|
|    cat|   tv|        1|
|   frog|table|        1|
|  eagle|glass|        1|
+-------+-----+---------+



## Chaining

cols y rows functions are used to organize and encapsulate optimus' functionality apart of Apache Spark Dataframe API. This have a disadvantage at chaining time because we need to user invoke cols or rows in every step.

At the same time it can be helpfull when you look at the code because every line is self explained.

In [20]:
df\
    .cols().rename([('num','number')])\
    .cols().drop(["number","words"])\
    .withColumn("new_col_2", lit("spongebob"))\
    .cols().create("new_col_1", 1)\
    .cols().sort(reverse= True)\
    .show()

+-----+---------+---------+-------+
|thing|new_col_2|new_col_1|animals|
+-----+---------+---------+-------+
|housé|spongebob|        1|    dog|
|   tv|spongebob|        1|    cat|
|table|spongebob|        1|   frog|
|glass|spongebob|        1|  eagle|
+-----+---------+---------+-------+



## Split Columns
### Spark

### Pandas

In [62]:
df.cols().split("two strings","-", n=3).show()

+-------------------+---+-------+-----+-------------+-----+-----+-----+
|              words|num|animals|thing|  two strings|COL_0|COL_1|COL_2|
+-------------------+---+-------+-----+-------------+-----+-----+-----+
|  I like     fish  |  1|    dog|housé|      cat-car|  cat|  car| null|
|            zombies|  2|    cat|   tv|       dog-tv|  dog|   tv| null|
|simpsons   cat lady|  2|   frog|table|eagle-tv-plus|eagle|   tv| plus|
|               null|  3|  eagle|glass|      lion-pc| lion|   pc| null|
+-------------------+---+-------+-----+-------------+-----+-----+-----+



In [59]:
df.cols().split("two strings","-", get = 1).show()

+-------------------+---+-------+-----+-------------+-----+
|              words|num|animals|thing|  two strings|COL_1|
+-------------------+---+-------+-----+-------------+-----+
|  I like     fish  |  1|    dog|housé|      cat-car|  car|
|            zombies|  2|    cat|   tv|       dog-tv|   tv|
|simpsons   cat lady|  2|   frog|table|eagle-tv-plus|   tv|
|               null|  3|  eagle|glass|      lion-pc|   pc|
+-------------------+---+-------+-----+-------------+-----+



## Filter

In [68]:
df.show()

+-------------------+---+-------+-----+-------------+------+
|              words|num|animals|thing|  two strings|filter|
+-------------------+---+-------+-----+-------------+------+
|  I like     fish  |  1|    dog|housé|      cat-car|     a|
|            zombies|  2|    cat|   tv|       dog-tv|     b|
|simpsons   cat lady|  2|   frog|table|eagle-tv-plus|     1|
|               null|  3|  eagle|glass|      lion-pc|     c|
+-------------------+---+-------+-----+-------------+------+



In [82]:
df.rows().filter("filter", type = "integer").show()

filter
+-------------------+---+-------+-----+-----------+------+
|              words|num|animals|thing|two strings|filter|
+-------------------+---+-------+-----+-----------+------+
|  I like     fish  |  1|    dog|housé|    cat-car|     a|
|            zombies|  2|    cat|   tv|     dog-tv|     b|
|               null|  3|  eagle|glass|    lion-pc|     c|
+-------------------+---+-------+-----+-----------+------+

